In [1]:
import pyspark
from pyspark.sql import SparkSession
pyspark.__version__

'2.3.2'

In [2]:
# 스파크 context객체 가져오기
spark = SparkSession.builder.getOrCreate()
# spark=SparkContext() : 1점대 버전에선 이렇게 한다.

In [13]:
df = spark.read.json("/home/tutor/data/simple-ml") # 비정형데이터를 많이 다루다보니 json데이터를 많이 사용한다.
df   # dataFrame은 저장단위의 데이터이고 얘를 가공한 형태로 쓰려면 .show()

DataFrame[color: string, lab: string, value1: bigint, value2: double]

In [12]:
df.show()

+-----+----+------+------------------+
|color| lab|value1|            value2|
+-----+----+------+------------------+
|green|good|     1|14.386294994851129|
| blue| bad|     8|14.386294994851129|
| blue| bad|    12|14.386294994851129|
|green|good|    15| 38.97187133755819|
|green|good|    12|14.386294994851129|
|green| bad|    16|14.386294994851129|
|  red|good|    35|14.386294994851129|
|  red| bad|     1| 38.97187133755819|
|  red| bad|     2|14.386294994851129|
|  red| bad|    16|14.386294994851129|
|  red|good|    45| 38.97187133755819|
|green|good|     1|14.386294994851129|
| blue| bad|     8|14.386294994851129|
| blue| bad|    12|14.386294994851129|
|green|good|    15| 38.97187133755819|
|green|good|    12|14.386294994851129|
|green| bad|    16|14.386294994851129|
|  red|good|    35|14.386294994851129|
|  red| bad|     1| 38.97187133755819|
|  red| bad|     2|14.386294994851129|
+-----+----+------+------------------+
only showing top 20 rows



In [15]:
df.orderBy("value2").show()  #value2 기준으로 정렬

+-----+----+------+------------------+
|color| lab|value1|            value2|
+-----+----+------+------------------+
|green|good|     1|14.386294994851129|
|green| bad|    16|14.386294994851129|
| blue| bad|     8|14.386294994851129|
| blue| bad|     8|14.386294994851129|
| blue| bad|    12|14.386294994851129|
|green| bad|    16|14.386294994851129|
|green|good|    12|14.386294994851129|
|  red|good|    35|14.386294994851129|
|  red|good|    35|14.386294994851129|
|  red| bad|     2|14.386294994851129|
|  red| bad|    16|14.386294994851129|
|  red| bad|    16|14.386294994851129|
| blue| bad|     8|14.386294994851129|
|green|good|     1|14.386294994851129|
|green|good|    12|14.386294994851129|
| blue| bad|     8|14.386294994851129|
|  red|good|    35|14.386294994851129|
| blue| bad|    12|14.386294994851129|
|  red| bad|    16|14.386294994851129|
|green|good|    12|14.386294994851129|
+-----+----+------+------------------+
only showing top 20 rows



In [27]:
df.select('lab').filter("lab=='good'").show()

+----+
| lab|
+----+
|good|
|good|
|good|
|good|
|good|
|good|
|good|
|good|
|good|
|good|
|good|
|good|
|good|
|good|
|good|
|good|
|good|
|good|
|good|
|good|
+----+
only showing top 20 rows



In [43]:
# 교재 635
bInput = spark.read.format("parquet").load("./data/binary-classification").selectExpr("features", "cast(label as double) as label")
# mllibrary의특징: 

In [39]:
bInput

DataFrame[features: vector, label: double]

In [44]:
bInput.show()   #mllibrary가 학습할 수 있도록 생김

+--------------+-----+
|      features|label|
+--------------+-----+
|[3.0,10.1,3.0]|  1.0|
|[1.0,0.1,-1.0]|  0.0|
|[1.0,0.1,-1.0]|  0.0|
| [2.0,1.1,1.0]|  1.0|
| [2.0,1.1,1.0]|  1.0|
+--------------+-----+



In [46]:
from pyspark.ml.classification import LogisticRegression

In [49]:
lr = LogisticRegression()
lr

LogisticRegression_439887e9fd8fa27406ae

In [51]:
print(lr.explainParams()) # see all parameters

aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)
family: The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial (default: auto)
featuresCol: features column name. (default: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label)
lowerBoundsOnCoefficients: The lower bounds on coefficients if fitting under bound constrained optimization. The bound matrix must be compatible with the shape (1, number of features) for binomial regression, or (number of classes, number of features) for multinomial regression. (undefined)
lowerBoundsOnIntercepts: The lower bounds on intercepts if fitting under bound constrained optimization. The bounds vector size must beequal wi

In [52]:
lrModel = lr.fit(bInput)

In [53]:
lrModel.coefficients  # 계수값

DenseVector([6.8487, 0.3536, 14.8149])

In [54]:
lrModel.intercept  # 절편
# 스파크는 전처리하기 전까지만 쓴다. 데이터 수집하고 저장. (크롤링 빨리 잘해준다.) 모형은 그닥...

-10.225695864480924

In [59]:
summary = lrModel.summary
print(summary.areaUnderROC)
summary.roc.show()
summary.pr.show() #FPR:거짓률.틀린것.  TPR:정답률.맞은것  recall: precision:정밀도.예측치 기준으로 실제로 맞춘 게 몇개?

1.0
+---+------------------+
|FPR|               TPR|
+---+------------------+
|0.0|               0.0|
|0.0|0.3333333333333333|
|0.0|               1.0|
|1.0|               1.0|
|1.0|               1.0|
+---+------------------+

+------------------+---------+
|            recall|precision|
+------------------+---------+
|               0.0|      1.0|
|0.3333333333333333|      1.0|
|               1.0|      1.0|
|               1.0|      0.6|
+------------------+---------+

